In [1]:
import pandas as pd
import numpy as np

def load_dataframe(relative_path,dataframe_name):
    df = pd.read_pickle(f'{relative_path}/{dataframe_name}.pkl')    
    return df

def read_file(relative_path,file_name):
    text= ""
    with open(f'{relative_path}/{file_name}.abc','r') as f:
        text = f.read()
    return text

In [2]:
relative_path ="notebooks/data/augmented_dataset"
filename_name = 'clean_augmented_data'
#filename_name = 'clean_original_training_data'
#relative_path ="notebooks/data/original_dataset"
training_data_df = load_dataframe(relative_path,filename_name)
training_data_df.columns


Index(['unit_note_length', 'tuneBook', 'title', 'reference_number',
       'original_header', 'original_body', 'meter', 'key', 'clean_song',
       'clean_header', 'clean_body', 'chord_progression', '"Gm"', '"G7"',
       '"G"', '"F7"', '"F#m"', '"F#7"', '"F"', '"Em"', '"Eb"', '"E7"', '"E"',
       '"Dm"', '"D7"', '"D"', '"Cm"', '"C7"', '"C"', '"Bm"', '"Bb"', '"B7"',
       '"B"', '"Am"', '"A7"', '"A"'],
      dtype='object')

In [3]:
training_data_df.head()["clean_header"]

0    M:3/2\nL:1/8\nK:C#m\n|"Gm""D7""Gm"|"Cm""D7""Gm...
1    M:4/4\nL:1/4\nK:Db\n|"G"|"D7"|"G"|"Am""D7"|"G"...
2    M:6/8\nL:1/8\nK:Db\n|"G"|"G"|"G7"|"C"|"G"|"G"|...
3    M:4/4\nL:1/4\nK:G#m\n|"Dm"|"Dm"|"Dm"|"Dm"|"F"|...
4    M:6/8\nL:1/8\nK:Ab\n|"D"|"G""A7"|"D""Em"|"E7""...
Name: clean_header, dtype: object

In [4]:
training_data_df.head()["clean_body"]

0    "Gm"g4"D7"^f2=ef"Gm"g4|"Cm"d2cB"D7"ABc2"Gm"B2G...
1    |D|"G"GBBA/2B/2|"D7"cAAd|"G"BGGB|"Am"A/2G/2F/2...
2    "G"DEDG2A|"G"BcBA2G|"G7"dB2A2G|"C"E3-E3|"G"DED...
3    a|"Dm"afga|"Dm"fe/2f/2da|"Dm"afga|"Dm"f2fa|"F"...
4    A|"D"d2efdf|"G"g3"A7"fga|"D"fgf"Em"e2d|"E7"e3"...
Name: clean_body, dtype: object

In [5]:
training_data_df.tail()

,unit_note_length,tuneBook,title,reference_number,original_header,original_body,meter,key,clean_song,clean_header,...,"""Cm""","""C7""","""C""","""Bm""","""Bb""","""B7""","""B""","""Am""","""A7""","""A"""
19445,1/4,dataset_min5.abc,Grandpa's,78,X:78\nT:Grandpa's\nM:4/4\nL:1/4\nK:A,"A/2G/2|""D""FA""A7""Bc|""D""d/2c/2d/2e/2fa|""Em""gf""E7...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""|""Em""""E7""|""A7""|...","M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""|""Em""""E7""|""A7""|...",...,0,0,0,0,0,0,0,0,7,2
19446,1/4,dataset_min5.abc,The Girl With The Green Hat On,79,X:79\nT:The Girl With The Green Hat On\nM:4/4\...,"(3A/2B/2c/2|""D""dA""A7""A/2B/2A/2G/2|""D""F/2G/2A/2...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""""A7""|""D""|""A7""|""...","M:4/4\nL:1/4\nK:A\n|""D""""A7""|""D""""A7""|""D""|""A7""|""...",...,0,0,0,0,0,0,0,0,6,2
19447,1/4,dataset_min5.abc,Green Meadow,80,X:80\nT:Green Meadow\nM:4/4\nL:1/4\nK:D,"(3D/2E/2F/2|""G""GG/2A/2B/2G/2B/2d/2|""C""e/2f/2g/...",4/4,D,"M:4/4\nL:1/4\nK:D\n|""G""|""C""""G""|""Am""|""Am""""D7""|""...","M:4/4\nL:1/4\nK:D\n|""G""|""C""""G""|""Am""|""Am""""D7""|""...",...,0,0,6,0,0,0,0,4,0,0
19448,1/4,dataset_min5.abc,The Old Grey Cat,82,X:82\nT:The Old Grey Cat\nM:4/4\nL:1/4\nK:Bm,"B|""Em""eeEE/2F/2|""Em""G/2F/2G/2A/2B/2A/2B/2^c/2|...",4/4,Bm,"M:4/4\nL:1/4\nK:Bm\n|""Em""|""Em""|""D""|""D""|""Em""|""E...","M:4/4\nL:1/4\nK:Bm\n|""Em""|""Em""|""D""|""D""|""Em""|""E...",...,0,0,0,0,0,3,0,1,0,0
19449,1/4,dataset_min5.abc,Gyre And Gimble,84,X:84\nT:Gyre And Gimble\nM:4/4\nL:1/4\nK:A,"A|""D""dAFA|""Em""BG""A7""EG|""D""FAd3/2e/2|""A7""f/2g/2...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""D""|""Em""""A7""|""D""|""A7""""D""|""...","M:4/4\nL:1/4\nK:A\n|""D""|""Em""""A7""|""D""|""A7""""D""|""...",...,0,0,0,0,0,0,0,0,7,1


In [6]:
training_data_df["clean_header"].str.len().max()

644

In [7]:
bodies = ""
silences = 0
for body in training_data_df["clean_body"]:
    if 'z' in body:
        silences +=1 
    bodies += body+"\n"
chars = sorted(list(set(bodies)))
vocab_size = len(chars)
print('vocab: ',''.join(chars))
print('vocab_size',vocab_size)
print("silences ",silences)

vocab:  
"#'()+,-/1234567=ABCDEFG[]^_abcdefgmz|~
vocab_size 40
silences  1224


In [8]:
training_data_text = read_file(relative_path,filename_name)

print("number of chars:",len(training_data_text))

number of chars: 10352974


In [9]:
chars = sorted(list(set(training_data_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


"#'()+,-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
46


In [10]:
import torch

print(torch.__version__)

2.0.0+cu117


In [11]:
import transformers
print(transformers.__version__)

4.28.0.dev0


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import wandb
import tiktoken

print(wandb.__version__)

0.14.2


In [13]:
!pip freeze

absl-py==1.4.0
aiohttp==3.8.4
aiosignal==1.3.1
alembic==1.10.2
antlr4-python3-runtime==4.9.3
anyio==3.6.2
appdirs==1.4.4
APScheduler==3.10.1
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
async-timeout==4.0.2
attrs==22.2.0
audioread==3.0.0
av==9.2.0
Babel==2.12.1
backcall==0.2.0
backoff==1.11.1
backports.zoneinfo==0.2.1
beautifulsoup4==4.11.2
binaryornot==0.4.4
black==23.1.0
bleach==6.0.0
cachetools==5.3.0
certifi==2022.12.7
cffi==1.15.1
chardet==5.1.0
charset-normalizer==3.1.0
click==8.1.3
clldutils==3.19.0
cloudpickle==2.2.1
cmaes==0.9.1
cmake==3.25.0
codecarbon==1.2.0
colorama==0.4.6
coloredlogs==15.0.1
colorlog==6.7.0
comm==0.1.3
contourpy==1.0.7
cookiecutter==1.7.3
csvw==3.1.3
cycler==0.11.0
dash==2.8.1
dash-bootstrap-components==1.4.0
dash-core-components==2.0.0
dash-html-components==2.0.0
dash-table==5.0.0
datasets==2.10.1
debugpy==1.6.7
decorator==5.1.1
decord==0.6.0
defusedxml==0.7.1
detectron2 @ git+https://github.com/facebookresearch/detectron

In [14]:
!ls

Dockerfile  docker-compose.yaml  overrides.json
README.md   notebooks		 requirements.txt


In [15]:
import os

nano_path = 'notebooks/nanoGPT'
os.chdir(nano_path)
!ls

LICENSE      bench.py	      model.py		    train.py
README.md    config	      out-shakespeare-char  transformer_sizing.ipynb
__pycache__  configurator.py  sample.py
assets	     data	      scaling_laws.ipynb


In [16]:
!python3 data/shakespeare_char/prepare.py

length of dataset in characters: 10,352,974
all the unique characters: 
"#'()+,-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
vocab size: 46
train has 9,317,676 tokens
val has 1,035,298 tokens


In [17]:
!python3 train.py config/train_shakespeare_char.py

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 50 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = True

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 500
lr_decay_iters = 500 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_ite

In [18]:
 !python3 sample.py --out_dir=out-shakespeare-char

Overriding: out_dir = out-shakespeare-char
number of parameters: 10.64M
Loading meta from data/shakespeare_char/meta.pkl...

K:C
|"D""A7"|"D""A7"|"D"|"D""G"|"D""A7"|"D"|"D""A7"|"D"|"D"|"D""A7"|"D"|"D""A7"|"D"|"D"|"G""A7"|"D"|"A7"|"A7"|"D""A7"|"D"|"D""A7"|"G""D7"|"A7"|"D""A7"|"D""A7"|"D"|"D""G""A7"|"D""A7"|"D""A7"|"D""A7"|"D""A7"|"D"|"A7"|"A7"|"D""A"|"D"|]
|"A7""D"|"D"|"D"|"A7""A7"|"D"|"G""A7"|"D"|"A7"|"A7"|"D"|"E7"|"A7"|"D""A"|"A7"|"D"|"D""A7"|"D"|"A7"|]A/2A/2|"D"d/2e/4d/4|"D"f/2e/2d/2|"G"B/2A/2D/2|"D"c/2d/2|"D"d/2c/2d/2d/2"A7"e/2"D"d/2c/2|"D"c/2"A7"c/2B/2G/2A/2|"D"F/2G/2A/2|"D"d/2d/2d/2d/2|"G"B/2d/2B/2B/2G/2A/2B/2|"D
---------------

|"Em"|"E7"|"Am"|"Em"|"Am"|"Am"|"Em"|"Em"|"Am"|"E7"|"Am"|"Em"|"Em"|"Em"|"Am"|"E7"|"Am"|"Am"|"Em"|"Am"|"E7"|"Am"|"A7"|"Am"|"Am"|"Am"|"E7"|"Em"|"Am"|"Em"|"Em"|"Am"|"Am"|"E7"|"Am"|"Am"|"Am"|"Am"|"E7"|"E7"|]
|"Am"|"Am"|"Am"|"Am"|"E7"|"Em"|"Am"|"Am"|"Am"|"Am"|"Em"|"Am"|"D7"|]AA|"E7"A2d|"Am"e2BAB|"Am"e2BB|"Am"A2d|"Am"e2GABA|"Am"A2Bc|"E7"B2^^AAe|"Am"efeA"Am"e^G|"